In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True' # only potentially necessary if you have MacBook with M1/M2 chip

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import copy
import matplotlib.pyplot as plt

In [ ]:
# download MNIST data

# download training set
mnist_training = torchvision.datasets.MNIST('./', train=True, download=True, transform=transforms.ToTensor())
# download test set
mnist_testing = torchvision.datasets.MNIST('./', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
# subsample datasets
N_training_examples = 5000 # roughly 500 per class
N_validation_examples = 1000 # roughly 100 per class
# grab indices of training and validation examples
all_indices = np.random.choice(np.arange(len(mnist_training)),
                               size=N_training_examples+N_validation_examples)
training_indices = all_indices[:N_training_examples]
validation_indices = all_indices[N_training_examples:]

In [ ]:
# model code
class FullyConnected(nn.Module):
    # constructor
    def __init__(self, layer_widths=[784, 10], nonlinearity=nn.ReLU()):
        super(FullyConnected, self).__init__()
        layers = []
        for i in range(len(layer_widths)-2):
            layers.append(nn.Linear(layer_widths[i], layer_widths[i+1])) # fully connected layer
            layers.append(nonlinearity) # non-linearity
        # add final layer
        layers.append(nn.Linear(layer_widths[-2], layer_widths[-1]))
        self.model = nn.Sequential(*layers)
    
    # forward pass
    def forward(self, x):
        # input is passed in series to each module in the nn.Sequential object, i.e. each of the layers
        return self.model(x)

# example model with one hidden layer, hyperbolic tangent non-linearity
model = FullyConnected([784, 100, 10], nn.Tanh())
print(model.model)

# example model with two hidden layers, ReLU non-linearity
model = FullyConnected([784, 128, 64, 10])
print(model.model)

In [ ]:
def train(model, training_data, training_indices, validation_indices, config, verbose=False):
    # unpack configuration parameters
    lr = config['lr'] # learning rate
    n_epochs = config['n_epochs'] # number of passes (epochs) through the training data
    batch_size = config['batch_size']
    
    # set up optimizer and loss function
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss() # cross-entropy loss
    
    # set up dataloaders
    train_sampler = torch.utils.data.SubsetRandomSampler(training_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(validation_indices)
    trainloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, sampler=train_sampler)
    valloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, sampler=val_sampler)
    
    # training loop
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    best_val_accuracy = 0
    best_model = None
    for n in range(n_epochs):
        # set model to training mode (unnecessary for this model, but good practice)
        model.train()
        epoch_loss = 0
        epoch_acc = 0
        for images, targets in trainloader:
            images = images.view(-1, 784) # flatten images
            optimizer.zero_grad() # zero out gradients
            class_logits = model(images)
            loss = criterion(class_logits, targets)
            loss.backward() # backpropagate to compute gradients
            optimizer.step() # update parameters using stochastic gradient descent
            # update epoch statistics
            epoch_loss += loss.item() # batch loss
            epoch_acc += (class_logits.data.max(1)[1]).eq(targets).sum().item() # number of correct predictions
            
        # validation
        epoch_loss /= len(trainloader)
        epoch_acc /= len(training_indices)
        val_loss, val_acc = validate(model, valloader, criterion)
        val_loss /= len(valloader)
        val_acc /= len(validation_indices)
        
        # log epoch information
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        # save best model, if necessary
        if val_acc > best_val_accuracy:
            best_model = copy.deepcopy(model)
        if verbose and (n+1) % (int(n_epochs/20)) == 0:
            print('Epoch {}/{}: (Train) Loss = {:.4e}, Acc = {:.4f}, (Val) Loss = {:.4e}, Acc = {:.4f}'.format(
                   n+1,
                   n_epochs,
                   epoch_loss,
                   epoch_acc,
                   val_loss,
                   val_acc))
        
        
    return (np.array(train_losses),
            np.array(train_accuracies),
            np.array(val_losses),
            np.array(val_accuracies),
            best_model)
        
def validate(model, dataloader, criterion):
    val_loss = 0
    val_acc = 0
    # set model to eval mode (again, unnecessary here but good practice)
    model.eval()
    # don't compute gradients since we are not updating the model, saves a lot of computation
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.view(-1, 784)
            class_logits = model(images)
            loss = criterion(class_logits, targets)
            val_loss += loss.item()
            val_acc += (class_logits.data.max(1)[1]).eq(targets).sum().item()
    return val_loss, val_acc

def test(model, dataset):
    # test best model on withheld test data
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=128)
    test_acc = 0
    model.eval()
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.view(-1, 784)
            class_logits = model(images)
            test_acc += (class_logits.data.max(1)[1]).eq(targets).sum().item()
    return test_acc/len(dataset)

In [ ]:
# putting it all together

# data
N_training_examples = 1000 
N_validation_examples = 1000 # 100 examples per class
random_seed = 1 # random seed for reproducibility
np.random.seed(random_seed)
all_indices = np.random.choice(np.arange(len(mnist_training)),
                               size=N_training_examples+N_validation_examples)
training_indices = all_indices[:N_training_examples]
validation_indices = all_indices[N_training_examples:]

# model
layer_widths = [784, 100, 10] # one hidden layer
nonlinearity = nn.ReLU() # ReLU nonlinearity
model = FullyConnected(layer_widths=layer_widths, nonlinearity=nonlinearity)

# configuration parameters
config = {'lr': 1e-1,
          'n_epochs': 200,
          'batch_size': 128}

# train
verbose = True
train_losses, train_accs, val_losses, val_accs, best_model = train(model,
                                                                   mnist_training,
                                                                   training_indices,
                                                                   validation_indices,
                                                                   config,
                                                                   verbose=verbose)

In [ ]:
# inspect training results
plt.figure(figsize=(15, 10))
plt.subplot(221)
plt.semilogy(train_losses, color='royalblue')
plt.xlabel('Epoch')
plt.title('Training loss')
plt.grid(True)
plt.subplot(222)
plt.plot(train_accs, color='darkorange')
plt.xlabel('Epoch')
plt.title('Training accuracy')
plt.grid(True)
plt.subplot(223)
plt.plot(val_losses, color='royalblue')
plt.xlabel('Epoch')
plt.title('Validation loss')
plt.grid(True)
plt.subplot(224)
plt.plot(val_accs, color='darkorange')
plt.xlabel('Epoch')
plt.title('Validation accuracy')
plt.grid(True)

In [ ]:
# test best model
print('Test accuracy = {:.4f}'.format(test(best_model, mnist_testing)))